In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fw_v0.interface import vlstm_fw_torch
from src.vlstm_fw_v0.interface import vlstm_fw_cuda

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/vlstm_fw_v0/build.ninja...
Building extension module vlstm_fw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module vlstm_fw_v0...


## CUDA vLSTM forward

In [3]:
S = 16 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 100.
igs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = 0.01 * torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs, qs.shape, len(qs.view(-1))

(tensor([[[[0.0000, 0.1000, 0.2000, 0.3000],
           [0.4000, 0.5000, 0.6000, 0.7000],
           [0.8000, 0.9000, 1.0000, 1.1000],
           [1.2000, 1.3000, 1.4000, 1.5000],
           [1.6000, 1.7000, 1.8000, 1.9000],
           [2.0000, 2.1000, 2.2000, 2.3000],
           [2.4000, 2.5000, 2.6000, 2.7000],
           [2.8000, 2.9000, 3.0000, 3.1000],
           [3.2000, 3.3000, 3.4000, 3.5000],
           [3.6000, 3.7000, 3.8000, 3.9000],
           [4.0000, 4.1000, 4.2000, 4.3000],
           [4.4000, 4.5000, 4.6000, 4.7000],
           [4.8000, 4.9000, 5.0000, 5.1000],
           [5.2000, 5.3000, 5.4000, 5.5000],
           [5.6000, 5.7000, 5.8000, 5.9000],
           [6.0000, 6.1000, 6.2000, 6.3000]]]], device='cuda:0'),
 torch.Size([1, 1, 16, 4]),
 64)

In [5]:
# pytorch version
rs = vlstm_fw_torch(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
)
rs, rs.shape

(tensor([[[[8.1548e-05, 8.1548e-05, 8.1548e-05, 8.1548e-05],
           [4.4926e-04, 4.4926e-04, 4.4926e-04, 4.4926e-04],
           [9.0641e-04, 9.0641e-04, 9.0641e-04, 9.0641e-04],
           [1.3812e-03, 1.3812e-03, 1.3812e-03, 1.3812e-03],
           [1.8511e-03, 1.8511e-03, 1.8511e-03, 1.8511e-03],
           [2.3116e-03, 2.3116e-03, 2.3116e-03, 2.3116e-03],
           [2.7640e-03, 2.7640e-03, 2.7640e-03, 2.7640e-03],
           [3.2106e-03, 3.2106e-03, 3.2106e-03, 3.2106e-03],
           [3.6533e-03, 3.6533e-03, 3.6533e-03, 3.6533e-03],
           [4.0937e-03, 4.0937e-03, 4.0937e-03, 4.0937e-03],
           [4.5327e-03, 4.5327e-03, 4.5327e-03, 4.5327e-03],
           [4.9708e-03, 4.9708e-03, 4.9708e-03, 4.9708e-03],
           [5.4085e-03, 5.4085e-03, 5.4085e-03, 5.4085e-03],
           [5.8460e-03, 5.8460e-03, 5.8460e-03, 5.8460e-03],
           [6.2833e-03, 6.2833e-03, 6.2833e-03, 6.2833e-03],
           [6.7205e-03, 6.7205e-03, 6.7205e-03, 6.7205e-03]]]], device='cuda:0'),
 to

In [6]:
# pytorch version QKV product (To test if this is still correct after changes to the code.)
# at some point we have to compare to the vlstm_fw_torch version.
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[0.0010, 0.0010, 0.0010, 0.0010],
           [0.0035, 0.0035, 0.0035, 0.0035],
           [0.0061, 0.0061, 0.0061, 0.0061],
           [0.0086, 0.0086, 0.0086, 0.0086],
           [0.0112, 0.0112, 0.0112, 0.0112],
           [0.0138, 0.0138, 0.0138, 0.0138],
           [0.0163, 0.0163, 0.0163, 0.0163],
           [0.0189, 0.0189, 0.0189, 0.0189],
           [0.0214, 0.0214, 0.0214, 0.0214],
           [0.0240, 0.0240, 0.0240, 0.0240],
           [0.0266, 0.0266, 0.0266, 0.0266],
           [0.0291, 0.0291, 0.0291, 0.0291],
           [0.0317, 0.0317, 0.0317, 0.0317],
           [0.0342, 0.0342, 0.0342, 0.0342],
           [0.0368, 0.0368, 0.0368, 0.0368],
           [0.0394, 0.0394, 0.0394, 0.0394]]]], device='cuda:0'),
 torch.Size([1, 1, 16, 4]))

In [7]:
# cuda kernel
rs = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 4
blocksxy: 1-2, threads: 4-4, shared_mem in bytes: 2560
In Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


(tensor([[[[0.0010, 0.0010, 0.0010, 0.0010],
           [0.0035, 0.0035, 0.0035, 0.0035],
           [0.0061, 0.0061, 0.0061, 0.0061],
           [0.0086, 0.0086, 0.0086, 0.0086],
           [0.0112, 0.0112, 0.0112, 0.0112],
           [0.0138, 0.0138, 0.0138, 0.0138],
           [0.0163, 0.0163, 0.0163, 0.0163],
           [0.0189, 0.0189, 0.0189, 0.0189],
           [0.0214, 0.0214, 0.0214, 0.0214],
           [0.0240, 0.0240, 0.0240, 0.0240],
           [0.0266, 0.0266, 0.0266, 0.0266],
           [0.0291, 0.0291, 0.0291, 0.0291],
           [0.0317, 0.0317, 0.0317, 0.0317],
           [0.0342, 0.0342, 0.0342, 0.0342],
           [0.0368, 0.0368, 0.0368, 0.0368],
           [0.0394, 0.0394, 0.0394, 0.0394]]]], device='cuda:0'),
 torch.Size([1, 1, 16, 4]))

## PyTorch vLSTM forward - Normalization

Shows that the result is identical, whether we use rowwise stabilization or not. 
For the Kernel it is easier to implement the rowwise normalization, therefore use the rowwise normalization as default. (It is also somewhat conceptually easier to convey. We treat each timestep independently.)

In most cases due to the input (i think forget gate is meant here??) gate the maximum is the last input timestep (i.e. the element on the diagonal). Then the result of the stabilization is that there are almost always ones on the diagonal, and the other timesteps are decayed accordingly.

In [23]:
from src.vlstm_fw_v0.torch_impl import vlstm_fw_torch

In [24]:
# params
S = 8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [25]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
igs = torch.rand((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
igs2 = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1)
fgs = torch.rand((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs.shape, fgs.shape

(torch.Size([1, 1, 8, 4]), torch.Size([1, 1, 8, 1]))

In [26]:
rs = vlstm_fw_torch(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
    stabilize_rowwise=True,
)
rs, rs.shape

(tensor([[[[-2.5441,  0.7163,  0.4934, -0.1267],
           [-0.0950,  0.1215, -0.2144, -0.2080],
           [-1.1865,  0.5090, -0.2582, -0.4349],
           [ 0.6274, -0.0040, -1.7983,  1.1217],
           [-0.2278,  0.4122, -2.1547,  1.6022],
           [ 0.8848, -0.3820,  0.6012, -0.5168],
           [ 1.9075, -1.2873, -0.1571,  0.0968],
           [ 1.5725, -1.0859, -0.3932,  1.7004]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 4]))

In [27]:
rs = vlstm_fw_torch(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
    stabilize_rowwise=False,
)
rs, rs.shape

(tensor([[[[-2.5441,  0.7163,  0.4934, -0.1267],
           [-0.0950,  0.1215, -0.2144, -0.2080],
           [-1.1865,  0.5090, -0.2582, -0.4349],
           [ 0.6274, -0.0040, -1.7983,  1.1217],
           [-0.2278,  0.4122, -2.1547,  1.6022],
           [ 0.8848, -0.3820,  0.6012, -0.5168],
           [ 1.9075, -1.2873, -0.1571,  0.0968],
           [ 1.5725, -1.0859, -0.3932,  1.7004]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 4]))

In [ ]:
from src.vlstm_fw_v0.interface import vlstm_fw

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/vlstm_fw_v0/build.ninja...
Building extension module vlstm_fw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
